### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
###Open AI API Key and open source models --Llama3, Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key


'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'

In [3]:
!pip install langchain_groq --user

  Using cached langchain_groq-0.2.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached groq-0.13.1-py3-none-any.whl.metadata (14 kB)
  Using cached langchain_core-0.3.28-py3-none-any.whl.metadata (6.3 kB)
  Using cached langsmith-0.2.6-py3-none-any.whl.metadata (14 kB)
Using cached langchain_groq-0.2.2-py3-none-any.whl (14 kB)
Using cached groq-0.13.1-py3-none-any.whl (109 kB)
Using cached langchain_core-0.3.28-py3-none-any.whl (411 kB)
Using cached langsmith-0.2.6-py3-none-any.whl (325 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.116
    Uninstalling langsmith-0.1.116:
      Successfully uninstalled langsmith-0.1.116
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.38
    Uninstalling langchain-core-0.2.38:
      Successfully uninstalled langchain-core-0.2.38


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.2.16 requires langchain-core<0.3.0,>=0.2.38, but you have langchain-core 0.3.28 which is incompatible.
langchain 0.2.16 requires langsmith<0.2.0,>=0.1.17, but you have langsmith 0.2.6 which is incompatible.
langchain-chroma 0.1.3 requires langchain-core<0.3,>=0.1.40, but you have langchain-core 0.3.28 which is incompatible.
langchain-community 0.2.16 requires langchain-core<0.3.0,>=0.2.38, but you have langchain-core 0.3.28 which is incompatible.
langchain-community 0.2.16 requires langsmith<0.2.0,>=0.1.0, but you have langsmith 0.2.6 which is incompatible.
langchain-text-splitters 0.2.4 requires langchain-core<0.3.0,>=0.2.38, but you have langchain-core 0.3.28 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade

In [4]:
from langchain_groq import ChatGroq
model = ChatGroq(model = "Gemma2-9b-It", groq_api_key = groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002C53AF81D60>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002C53AF53020>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
!pip install langchain_core


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from langchain_core.messages import HumanMessage,SystemMessage
messages = [
    SystemMessage(content = "Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result = model.invoke(messages)


In [9]:
result

AIMessage(content='Here are a couple of ways to say "Hello, how are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, ça va ?** (Informal) \n\n\nLet me know if you have any other phrases you\'d like translated!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 21, 'total_tokens': 85, 'completion_time': 0.116363636, 'prompt_time': 0.000139979, 'queue_time': 0.019494041, 'total_time': 0.116503615}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ce405957-d6f9-44f8-a1b0-a9a21276b795-0', usage_metadata={'input_tokens': 21, 'output_tokens': 64, 'total_tokens': 85})

In [10]:
#we have output_parser in langchain_core That will be responsible to display the messages that is coming out side as the response from LLM model.
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(result)

'Here are a couple of ways to say "Hello, how are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, ça va ?** (Informal) \n\n\nLet me know if you have any other phrases you\'d like translated!\n'

In [11]:
###Using LCEL - chain the components
chain = model |parser
chain.invoke(messages)

'Here are a couple of ways to say "Hello, how are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, comment vas-tu ?** (Informal)\n\n\n\nLet me know if you\'d like other phrases or have more translations!\n'

In [12]:
##One-technique as prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate the followign into {language}:"

prompt = ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)

In [14]:
result = prompt.invoke({"language":"French", "text":"Hello"})

In [15]:
result.to_messages()

[SystemMessage(content='Translate the followign into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [ ]:
##Chaining together components with LCEL
chain = prompt | model | parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'

In [ ]:
#Langserve helps developers deploy Langchain runnables and chains as REST API
#This library is integrated with FastAPI and uses pydantic for data validation.
